## original data

In [2]:
import pandas as pd
import numpy as np

In [3]:
geo = pd.read_csv('../raw_data/NYCTAXI202004-202006_OD/NYCTAXI202004-202006.geo')
print(geo.shape)
geo.head( )

(263, 3)


,geo_id,type,coordinates
0,0,Polygon,"[[['-74.18445300', '40.69499600'], ['-74.18563..."
1,1,MultiPolygon,"[[[['-73.81768667', '40.61493300'], ['-73.8165..."
2,2,Polygon,"[[['-73.84240629', '40.86847474'], ['-73.84280..."
3,3,Polygon,"[[['-73.97464721', '40.71884235'], ['-73.97347..."
4,4,Polygon,"[[['-74.17225508', '40.56170484'], ['-74.17348..."


In [4]:
geo["type"].value_counts()

Polygon         240
MultiPolygon     23
Name: type, dtype: int64

In [5]:
rel = pd.read_csv('../raw_data/NYCTAXI202004-202006_OD/NYCTAXI202004-202006.rel')
print(rel.shape)
rel.head()

(69169, 5)


,rel_id,type,origin_id,destination_id,cost
0,0,geo,0,0,0.0
1,1,geo,0,1,31349.0
2,2,geo,0,2,33747.0
3,3,geo,0,3,17205.0
4,4,geo,0,4,14927.0


In [6]:
# od = pd.read_csv('../raw_data/NYCTAXI202004-202006_OD/NYCTAXI202004-202006.od').drop(["type"],axis=1)
# print(od.shape)
# od.head()

## agg od by time   -> data/NYCTAXI_OD_agg.csv

In [7]:
# od_agg = od.groupby(["origin_id","destination_id"]).agg({"flow":"sum"}).reset_index()

In [8]:
# od_agg.to_csv("data/NYCTAXI_OD_agg.csv",index=False)

## join od_agg and rel

In [9]:
od_agg = pd.read_csv("data/NYCTAXI_OD_agg.csv")
od_f = od_agg.copy(deep = True)

od_f["cost"] = rel["cost"]

In [10]:
od_f.to_csv("data/NYCTAXI_OD_f.csv",index=False)

In [12]:
od_f.head()

,origin_id,destination_id,flow,cost
0,0,0,0.0,0.0
1,0,1,0.0,31349.0
2,0,2,0.0,33747.0
3,0,3,0.0,17205.0
4,0,4,0.0,14927.0


## add location to the od_f

In [22]:
od_f_merge = pd.merge(od_f,geo,left_on="origin_id",right_on="geo_id",how="left").drop(["geo_id","type"],axis=1)\
    .rename(columns={"coordinates":"origin_coordinates"})
od_f_merge

,origin_id,destination_id,flow,cost,origin_coordinates
0,0,0,0.0,0.0,"[[['-74.18445300', '40.69499600'], ['-74.18563..."
1,0,1,0.0,31349.0,"[[['-74.18445300', '40.69499600'], ['-74.18563..."
2,0,2,0.0,33747.0,"[[['-74.18445300', '40.69499600'], ['-74.18563..."
3,0,3,0.0,17205.0,"[[['-74.18445300', '40.69499600'], ['-74.18563..."
4,0,4,0.0,14927.0,"[[['-74.18445300', '40.69499600'], ['-74.18563..."
...,...,...,...,...,...
69164,262,258,25.0,15740.0,"[[['-73.95268515', '40.77233940'], ['-73.95218..."
69165,262,259,32.0,5206.0,"[[['-73.95268515', '40.77233940'], ['-73.95218..."
69166,262,260,24.0,9612.0,"[[['-73.95268515', '40.77233940'], ['-73.95218..."
69167,262,261,1634.0,571.0,"[[['-73.95268515', '40.77233940'], ['-73.95218..."


In [23]:
od_f_merge = pd.merge(od_f_merge,geo,left_on="destination_id",right_on="geo_id",how="left").drop(["geo_id","type"],axis=1)\
    .rename(columns={"coordinates":"destination_coordinates"})
od_f_merge

,origin_id,destination_id,flow,cost,origin_coordinates,destination_coordinates
0,0,0,0.0,0.0,"[[['-74.18445300', '40.69499600'], ['-74.18563...","[[['-74.18445300', '40.69499600'], ['-74.18563..."
1,0,1,0.0,31349.0,"[[['-74.18445300', '40.69499600'], ['-74.18563...","[[[['-73.81768667', '40.61493300'], ['-73.8165..."
2,0,2,0.0,33747.0,"[[['-74.18445300', '40.69499600'], ['-74.18563...","[[['-73.84240629', '40.86847474'], ['-73.84280..."
3,0,3,0.0,17205.0,"[[['-74.18445300', '40.69499600'], ['-74.18563...","[[['-73.97464721', '40.71884235'], ['-73.97347..."
4,0,4,0.0,14927.0,"[[['-74.18445300', '40.69499600'], ['-74.18563...","[[['-74.17225508', '40.56170484'], ['-74.17348..."
...,...,...,...,...,...,...
69164,262,258,25.0,15740.0,"[[['-73.95268515', '40.77233940'], ['-73.95218...","[[['-73.83737956', '40.89392474'], ['-73.83804..."
69165,262,259,32.0,5206.0,"[[['-73.95268515', '40.77233940'], ['-73.95218...","[[['-73.90147053', '40.75963305'], ['-73.90175..."
69166,262,260,24.0,9612.0,"[[['-73.95268515', '40.77233940'], ['-73.95218...","[[['-74.01428923', '40.70454907'], ['-74.01419..."
69167,262,261,1634.0,571.0,"[[['-73.95268515', '40.77233940'], ['-73.95218...","[[[['-73.94354421', '40.78288052'], ['-73.9436..."
